In [1]:
# -*- coding: utf-8 -*-
from __future__ import unicode_literals

from dotenv import load_dotenv
import os, sys

load_dotenv('../../')
import pandas as pd
sys.path.append(os.getenv('WORK_ENV'))
sys.path.append(os.getenv('SQLAPP_MODULES'))
from web.datalayer.category_sql import CategorySql, CategoryOnPublisher, CategoryOnPublisherMapper, CategoryNameByCountry
from web.datalayer.publisher import Publisher
from web.datalayer.organization_publisher import OrganizationPublisher
from web.datalayer.organization import Organization
from sqlapp import SqlApp
from web.datalayer.repositories import CategorySqlRepository
import pandas as pd
from sqlalchemy import func, and_, case
s = SqlApp()
s.connect('postgres://postgres@localhost:5432/partoo', echo=True)
session = s.session
Q = session.query

/Users/benamer/playgrounds/sqlapp/venv/lib/python2.7/site-packages/sqlalchemy/sql/schema.py:1226: SAWarning: Unicode column 'None' has non-unicode default value 'fr' specified.
  self.default


In [2]:
repo = CategorySqlRepository(db_session=session)

In [3]:
mappings = repo._get_subset_filter_by_missing_mappings(missing_mappings=True)

2020-07-01 15:10:54,027 INFO sqlalchemy.engine.base.Engine select version()
2020-07-01 15:10:54,029 INFO sqlalchemy.engine.base.Engine {}
2020-07-01 15:10:54,037 INFO sqlalchemy.engine.base.Engine select current_schema()
2020-07-01 15:10:54,039 INFO sqlalchemy.engine.base.Engine {}
2020-07-01 15:10:54,046 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-07-01 15:10:54,048 INFO sqlalchemy.engine.base.Engine {}
2020-07-01 15:10:54,056 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-07-01 15:10:54,057 INFO sqlalchemy.engine.base.Engine {}
2020-07-01 15:10:54,069 INFO sqlalchemy.engine.base.Engine show standard_conforming_strings
2020-07-01 15:10:54,071 INFO sqlalchemy.engine.base.Engine {}
2020-07-01 15:10:54,080 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-07-01 15:10:54,082 INFO sqlalchemy.engine.base.Engine SELECT count(*) AS count_1 
FROM (SELECT DISTINCT ON (category_on_

/Users/benamer/playgrounds/sqlapp/venv/lib/python2.7/site-packages/sqlalchemy/orm/relationships.py:1788: SAWarning: On GoogleMyBusinessSpecificFields.publisher_state, 'passive_deletes' is normally configured on one-to-many, one-to-one, many-to-many relationships only.
  % self)
/Users/benamer/playgrounds/sqlapp/venv/lib/python2.7/site-packages/sqlalchemy/orm/relationships.py:1788: SAWarning: On JustacoteSpecificFields.publisher_state, 'passive_deletes' is normally configured on one-to-many, one-to-one, many-to-many relationships only.
  % self)
/Users/benamer/playgrounds/sqlapp/venv/lib/python2.7/site-packages/sqlalchemy/orm/relationships.py:1788: SAWarning: On Group.parent, 'passive_deletes' is normally configured on one-to-many, one-to-one, many-to-many relationships only.
  % self)
/Users/benamer/playgrounds/sqlapp/venv/lib/python2.7/site-packages/sqlalchemy/orm/relationships.py:1788: SAWarning: On Comment.parent, 'passive_deletes' is normally configured on one-to-many, one-to-one, 

In [4]:
s = Q(
    func.count(CategorySql.id).label('all'),
    func.count(CategorySql.id).filter(CategorySql.state == 'DEPRECATED').label('deprecated'),
).subquery()

In [5]:
m = Q(func.count(mappings.c.id).label('missing_mappings')).subquery()

In [6]:
r = Q(s, m).first()
res = dict(
    all=r.all,
    deprecated=r.deprecated,
    missing_mappings=r.missing_mappings
)
res

2020-07-01 15:10:56,051 INFO sqlalchemy.engine.base.Engine SELECT anon_1."all" AS anon_1_all, anon_1.deprecated AS anon_1_deprecated, anon_2.missing_mappings AS anon_2_missing_mappings 
FROM (SELECT count(category.id) AS "all", count(category.id) FILTER (WHERE category.state = %(state_1)s) AS deprecated 
FROM category) AS anon_1, (SELECT count(anon_3.id) AS missing_mappings 
FROM (SELECT anon_4.id AS id, anon_4.gmb_id AS gmb_id, anon_4.state AS state, anon_4.block_push_long_description_on_gmb AS block_push_long_description_on_gmb, anon_4.live_id AS live_id, anon_4.created_at AS created_at, anon_4.updated_at AS updated_at, anon_4.count AS count, anon_4.missing_mappings AS missing_mappings 
FROM (SELECT anon_5.id AS id, anon_5.gmb_id AS gmb_id, anon_5.state AS state, anon_5.block_push_long_description_on_gmb AS block_push_long_description_on_gmb, anon_5.live_id AS live_id, anon_5.created_at AS created_at, anon_5.updated_at AS updated_at, anon_5.count AS count, CASE WHEN (anon_5.count < %

{'all': 4006L, 'deprecated': 1L, 'missing_mappings': 1029L}

In [10]:
counts = repo.get_stats()
counts

2020-07-01 15:11:41,014 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-07-01 15:11:41,017 INFO sqlalchemy.engine.base.Engine SELECT count(*) AS count_1 
FROM (SELECT DISTINCT ON (category_on_publisher.publisher_name) category_on_publisher.id AS category_on_publisher_id, category_on_publisher.publisher_name AS category_on_publisher_publisher_name, category_on_publisher.label AS category_on_publisher_label, category_on_publisher.publisher_id AS category_on_publisher_publisher_id, category_on_publisher.created_at AS category_on_publisher_created_at, category_on_publisher.updated_at AS category_on_publisher_updated_at 
FROM category_on_publisher) AS anon_1
2020-07-01 15:11:41,019 INFO sqlalchemy.engine.base.Engine {}
2020-07-01 15:11:41,042 INFO sqlalchemy.engine.base.Engine SELECT anon_1."all" AS anon_1_all, anon_1.deprecated AS anon_1_deprecated, anon_2.missing_mappings AS anon_2_missing_mappings 
FROM (SELECT count(category.id) AS "all", count(category.id) FILTER (WHERE catego

{'all': 4006L, 'deprecated': 1L, 'missing_mappings': 1029L}

# HANDLING SESSION

In [9]:
# ROLLBACK
session.rollback()

2020-07-01 15:11:37,962 INFO sqlalchemy.engine.base.Engine ROLLBACK


In [ ]:
# COMMIT
session.commit()

In [ ]:
# CLOSE
session.close()